In [52]:
import os
import csv
import numpy as np
import scipy.io as sio

from sklearn.linear_model import RidgeClassifier
from sklearn.feature_selection import RFE
from nilearn import connectome

from scipy.spatial import distance


# Reading and computing the input data

# Selected pipeline
pipeline = 'cpac'

# Input data variables
root_folder = '../ABIDE/'
data_folder = os.path.join(root_folder, 'ABIDE_pcp/cpac/filt_noglobal')
phenotype = os.path.join(root_folder, 'ABIDE_pcp/Phenotypic_V1_0b_preprocessed1.csv')


def fetch_filenames(subject_IDs, file_type):

    """
        subject_list : list of short subject IDs in string format
        file_type    : must be one of the available file types

    returns:

        filenames    : list of filetypes (same length as subject_list)
    """

    import glob

    # Specify file mappings for the possible file types
    filemapping = {'func_preproc': '_func_preproc.nii.gz',
                   'rois_ho': '_rois_ho.1D'}

    # The list to be filled
    filenames = []

    # Fill list with requested file paths
    for i in range(len(subject_IDs)):
        os.chdir(data_folder)  # os.path.join(data_folder, subject_IDs[i]))
        try:
            filenames.append(glob.glob('*' + subject_IDs[i] + filemapping[file_type])[0])
        except IndexError:
            # Return N/A if subject ID is not found
            filenames.append('N/A')

    return filenames


# Get timeseries arrays for list of subjects
def get_timeseries(subject_list, atlas_name):
    """
        subject_list : list of short subject IDs in string format
        atlas_name   : the atlas based on which the timeseries are generated e.g. aal, cc200

    returns:
        time_series  : list of timeseries arrays, each of shape (timepoints x regions)
    """

    timeseries = []
    for i in range(len(subject_list)):
        subject_folder = os.path.join(data_folder, subject_list[i])
        ro_file = [f for f in os.listdir(subject_folder) if f.endswith('_rois_' + atlas_name + '.1D')]
        fl = os.path.join(subject_folder, ro_file[0])
        print("Reading timeseries file %s" %fl)
        timeseries.append(np.loadtxt(fl, skiprows=0))

    return timeseries


# Compute connectivity matrices
def subject_connectivity(timeseries, subject, atlas_name, kind, save=True, save_path=data_folder):
    """
        timeseries   : timeseries table for subject (timepoints x regions)
        subject      : the subject ID
        atlas_name   : name of the parcellation atlas used
        kind         : the kind of connectivity to be used, e.g. lasso, partial correlation, correlation
        save         : save the connectivity matrix to a file
        save_path    : specify path to save the matrix if different from subject folder

    returns:
        connectivity : connectivity matrix (regions x regions)
    """

    print("Estimating %s matrix for subject %s" % (kind, subject))

    if kind in ['tangent', 'partial correlation', 'correlation']:
        conn_measure = connectome.ConnectivityMeasure(kind=kind)
        connectivity = conn_measure.fit_transform([timeseries])[0]

    if save:
        subject_file = os.path.join(save_path, subject,
                                    subject + '_' + atlas_name + '_' + kind.replace(' ', '_') + '.mat')
        sio.savemat(subject_file, {'connectivity': connectivity})

    return connectivity


# Get the list of subject IDs
def get_ids(num_subjects=None):
    """

    return:
        subject_IDs    : list of all subject IDs
    """

    subject_IDs = np.genfromtxt(os.path.join(data_folder, 'subject_IDs.txt'), dtype=str)

    if num_subjects is not None:
        subject_IDs = subject_IDs[:num_subjects]

    return subject_IDs


# Get phenotype values for a list of subjects
def get_subject_score(subject_list, score):
    scores_dict = {}

    with open(phenotype) as csv_file:
        reader = csv.DictReader(csv_file)
        for row in reader:
            if row['SUB_ID'] in subject_list:
                scores_dict[row['SUB_ID']] = row[score]

    return scores_dict

# int cnt=0;
# Dimensionality reduction step for the feature vector using a ridge classifier
def feature_selection(matrix, labels, train_ind, fnum):
    """
        matrix       : feature matrix (num_subjects x num_features)
        labels       : ground truth labels (num_subjects x 1)
        train_ind    : indices of the training samples
        fnum         : size of the feature vector after feature selection

    return:
        x_data      : feature matrix of lower dimension (num_subjects x fnum)
    """

    estimator = RidgeClassifier()
    selector = RFE(estimator, n_features_to_select=fnum, step=100, verbose=1)
#     print(cnt+1)
#     cnt=cnt+1;
    featureX = matrix[train_ind, :]
    featureY = labels[train_ind]
    selector = selector.fit(featureX, featureY.ravel())
    x_data = selector.transform(matrix)

    print("Number of labeled samples %d" % len(train_ind))
    print("Number of features selected %d" % x_data.shape[1])

    return x_data


# Make sure each site is represented in the training set when selecting a subset of the training set
def site_percentage(train_ind, perc, subject_list):
    """
        train_ind    : indices of the training samples
        perc         : percentage of training set used
        subject_list : list of subject IDs

    return:
        labeled_indices      : indices of the subset of training samples
    """

    train_list = subject_list[train_ind]
    sites = get_subject_score(train_list, score='SITE_ID')
    unique = np.unique(list(sites.values())).tolist()
    site = np.array([unique.index(sites[train_list[x]]) for x in range(len(train_list))])

    labeled_indices = []

    for i in np.unique(site):
        id_in_site = np.argwhere(site == i).flatten()

        num_nodes = len(id_in_site)
        labeled_num = int(round(perc * num_nodes))
        labeled_indices.extend(train_ind[id_in_site[:labeled_num]])

    return labeled_indices


# Load precomputed fMRI connectivity networks
def get_networks(subject_list, kind, atlas_name="aal", variable='connectivity'):
    """
        subject_list : list of subject IDs
        kind         : the kind of connectivity to be used, e.g. lasso, partial correlation, correlation
        atlas_name   : name of the parcellation atlas used
        variable     : variable name in the .mat file that has been used to save the precomputed networks


    return:
        matrix      : feature matrix of connectivity networks (num_subjects x network_size)
    """

    all_networks = []
    for subject1 in subject_list:
        fl = f'../Datasets/all_fc_matrix_rois_ez/matrix_rois_ez_{subject1}.mat'
        try:  
            matrix = sio.loadmat(fl)[variable]
            all_networks.append(matrix)
        except FileNotFoundError:
            fl = f'../Datasets/all_fc_matrix_rois_ez/matrix_rois_ez_{50002}.mat'
            matrix = sio.loadmat(fl)[variable]
            all_networks.append(matrix)
            
            
    # all_networks=np.array(all_networks)

    idx = np.triu_indices_from(all_networks[0], 1)
#     print("idx :- ",idx)
    norm_networks = [np.arctanh(mat) if not np.all(np.abs(mat) == 1) else mat for mat in all_networks]
#     print("norm_networks : ",norm_networks)
    vec_networks = [mat[idx] for mat in norm_networks]
#     print("vec_networks : ",vec_networks)
    matrix = np.vstack(vec_networks)
#     print(len(matrix) , " matrix : ",matrix);
    return matrix


# Construct the adjacency matrix of the population from phenotypic scores
def create_affinity_graph_from_scores(scores, pd_dict):
    num_nodes = len(pd_dict[scores[0]]) 
    graph = np.zeros((num_nodes, num_nodes))

    for l in scores:
        label_dict = pd_dict[l]

        if l in ['AGE_AT_SCAN', 'FIQ']:
            for k in range(num_nodes):
                for j in range(k + 1, num_nodes):
                    try:
                        val = abs(float(label_dict[k]) - float(label_dict[j]))
                        if val < 2:
                            graph[k, j] += 1
                            graph[j, k] += 1
                    except ValueError:  # missing label
                        pass

        else:
            for k in range(num_nodes):
                for j in range(k + 1, num_nodes):
                    if label_dict[k] == label_dict[j]:
                        graph[k, j] += 1
                        graph[j, k] += 1

    return graph

def get_static_affinity_adj(features, pd_dict):
    pd_affinity = create_affinity_graph_from_scores(['SEX', 'SITE_ID'], pd_dict) 
    distv = distance.pdist(features, metric='correlation') 
    dist = distance.squareform(distv)  
    sigma = np.mean(dist)
    feature_sim = np.exp(- dist ** 2 / (2 * sigma ** 2))
    adj = pd_affinity * feature_sim  

    return adj


In [53]:
import numpy as np
import scipy.sparse as sp
import torch


from sklearn.model_selection import StratifiedKFold
from scipy.spatial import distance
from scipy.sparse.linalg.eigen import eigsh


def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot

def sample_mask(idx, l):
    """Create mask."""
    mask = np.zeros(l)
    mask[idx] = 1
    return np.array(mask, dtype=np.bool)

def get_train_test_masks(labels, idx_train, idx_val, idx_test):
    train_mask = sample_mask(idx_train, labels.shape[0])
    val_mask = sample_mask(idx_val, labels.shape[0])
    test_mask = sample_mask(idx_test, labels.shape[0])

    y_train = np.zeros(labels.shape)
    y_val = np.zeros(labels.shape)
    y_test = np.zeros(labels.shape)
    y_train[train_mask, :] = labels[train_mask, :]
    y_val[val_mask, :] = labels[val_mask, :]
    y_test[test_mask, :] = labels[test_mask, :]

    return y_train, y_val, y_test, train_mask, val_mask, test_mask

def load_data(subject_IDs, params): 
    
    # labels
    num_classes = 2
    num_nodes = len(subject_IDs)
    
    # 初始化y_data(), y
    y_data = np.zeros([num_nodes, num_classes])
    y = np.zeros([num_nodes, 1])
    
    labels = get_subject_score(subject_IDs, score='DX_GROUP')
    features = get_networks(subject_IDs, kind=params['connectivity'], atlas_name=params['atlas'])
    
    for i in range(num_nodes):
        y_data[i, int(labels[subject_IDs[i]]) - 1] = 1 # (871,2)
        y[i] = int(labels[subject_IDs[i]]) # (871,)
        
    skf = StratifiedKFold(n_splits=10)
    cv_splits = list(skf.split(features, np.squeeze(y)))
    train = cv_splits[params['folds']][0]
    test = cv_splits[params['folds']][1]
    val = test
    
    print('Number of train sample:{}' .format(len(train)))
        
    y_train, y_val, y_test, train_mask, val_mask, test_mask = get_train_test_masks(y_data, train, val, test)
    
    y_data = torch.LongTensor(np.where(y_data)[1])
    y = torch.LongTensor(y)
    y_train = torch.LongTensor(y_train[1])
    y_val = torch.LongTensor(y_val[1])
    y_test = torch.LongTensor(y_test[1])
    
    train = torch.LongTensor(train)
    val = torch.LongTensor(val)
    test = torch.LongTensor(test)
    train_mask = torch.LongTensor(train_mask)
    val_mask = torch.LongTensor(val_mask)
    test_mask = torch.LongTensor(test_mask)
    
    # Eigenvector
    labeled_ind = site_percentage(train, params['num_training'], subject_IDs)
    x_data = feature_selection(features, y, labeled_ind, params['num_features'])
    features = preprocess_features(sp.coo_matrix(x_data).tolil())
    features = torch.FloatTensor(np.array(features.todense()))
    
    # Adjacency matrix
    graph = create_affinity_graph_from_scores(['SEX', 'SITE_ID'], subject_IDs)
    distv = distance.pdist(x_data, metric='correlation')
    dist = distance.squareform(distv)
    sigma = np.mean(dist)
    sparse_graph = np.exp(- dist ** 2 / (2 * sigma ** 2))
    final_graph = graph * sparse_graph

    return final_graph, features, y, y_data, y_train, y_val, y_test, train, val, test, train_mask, val_mask, test_mask


def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def sparse_to_tuple(sparse_mx):
    """Convert sparse matrix to tuple representation."""
    def to_tuple(mx):
        if not sp.isspmatrix_coo(mx):
            mx = mx.tocoo()
        coords = np.vstack((mx.row, mx.col)).transpose()
        values = mx.data
        shape = mx.shape
        coords = torch.from_numpy(coords)
        values = torch.from_numpy(values)
        shape = torch.tensor(shape)
        return coords, values, shape

    if isinstance(sparse_mx, list):
        for i in range(len(sparse_mx)):
            sparse_mx[i] = to_tuple(sparse_mx[i])
    else:
        sparse_mx = to_tuple(sparse_mx)

    return sparse_mx

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)


def preprocess_features(features):
    """Row-normalize feature matrix"""
    rowsum = np.array(features.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    features = r_mat_inv.dot(features)
    return features

def normalize_adj(adj):
    """Symmetrically normalize adjacency matrix."""
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()

def preprocess_adj(adj):
    """Preprocessing of adjacency matrix for simple GCN model."""
    adj_normalized = normalize_adj(adj + sp.eye(adj.shape[0]))
    return adj_normalized

def chebyshev_polynomials(adj, k):
    """Calculate Chebyshev polynomials up to order k. Return a list of sparse matrices (tuple representation)."""
    print("Calculating Chebyshev polynomials up to order {}...".format(k))

    adj_normalized = normalize_adj(adj)
    laplacian = sp.eye(adj.shape[0]) - adj_normalized
    largest_eigval, _ = eigsh(laplacian, 1, which='LM')
    scaled_laplacian = (2. / largest_eigval[0]) * laplacian - sp.eye(adj.shape[0])

    t_k = list()
    t_k.append(sp.eye(adj.shape[0]))
    t_k.append(scaled_laplacian)

    def chebyshev_recurrence(t_k_minus_one, t_k_minus_two, scaled_lap):
        s_lap = sp.csr_matrix(scaled_lap, copy=True)
        return 2 * s_lap.dot(t_k_minus_one) - t_k_minus_two

    for i in range(2, k+1):
        t_k.append(chebyshev_recurrence(t_k[-1], t_k[-2], scaled_laplacian))

    return t_k



C:\Users\KSB\AppData\Local\Temp\ipykernel_2032\3119403776.py:8: DeprecationWarning: Please use `eigsh` from the `scipy.sparse.linalg` namespace, the `scipy.sparse.linalg.eigen` namespace is deprecated.
  from scipy.sparse.linalg.eigen import eigsh


In [54]:
import numpy as np
import torch
# from utils import preprocess_features
from sklearn.model_selection import StratifiedKFold


class dataloader():
    def __init__(self): 
        self.pd_dict = {}
        self.node_ftr_dim = 2000
        self.num_classes = 2 

    def load_data(self, params, connectivity='correlation', atlas='ho'):
        ''' load multimodal data from ABIDE
        return: imaging features (raw), labels, non-image data
        '''
        subject_IDs = get_ids()
        labels = get_subject_score(subject_IDs, score='DX_GROUP')
        num_nodes = len(subject_IDs)

        sites = get_subject_score(subject_IDs, score='SITE_ID')
        unique = np.unique(list(sites.values())).tolist()
        ages = get_subject_score(subject_IDs, score='AGE_AT_SCAN')
        genders = get_subject_score(subject_IDs, score='SEX') 

        y_onehot = np.zeros([num_nodes, self.num_classes])
        y = np.zeros([num_nodes])
        site = np.zeros([num_nodes], dtype=int)
        age = np.zeros([num_nodes], dtype=np.float32)
        gender = np.zeros([num_nodes], dtype=int)
        for i in range(num_nodes):
            y_onehot[i, int(labels[subject_IDs[i]])-1] = 1
            y[i] = int(labels[subject_IDs[i]])
            site[i] = unique.index(sites[subject_IDs[i]])
            age[i] = float(ages[subject_IDs[i]])
            gender[i] = genders[subject_IDs[i]]
        
        self.y = y -1  

        self.raw_features = get_networks(subject_IDs, kind=connectivity, atlas_name=atlas)

        phonetic_data = np.zeros([num_nodes, 3], dtype=np.float32)
        phonetic_data[:,0] = site 
        phonetic_data[:,1] = gender 
        phonetic_data[:,2] = age 

        self.pd_dict['SITE_ID'] = np.copy(phonetic_data[:,0])
        self.pd_dict['SEX'] = np.copy(phonetic_data[:,1])
        self.pd_dict['AGE_AT_SCAN'] = np.copy(phonetic_data[:,2]) 
        
        return self.raw_features, self.y, phonetic_data

    def data_split(self, n_folds):
        # split data by k-fold CV
        skf = StratifiedKFold(n_splits=n_folds)
        print("skf : ",skf)
        cv_splits = list(skf.split(self.raw_features, self.y))
#         print("cv_splits : ",len(cv_splits))
#         x=len(cv_splits);
#         for i in range(x):
#             y=len(cv_splits[i]);
#             for j in range(y):
#                 z=len(cv_splits[i][j])
#                 print(z)
            
#         print("cv_splits : ",cv_splits  )
        return cv_splits 

    def get_node_features(self, train_ind):
        '''preprocess node features for wl-deepgcn
        '''
        node_ftr = feature_selection(self.raw_features, self.y, train_ind, self.node_ftr_dim)
        self.node_ftr = preprocess_features(node_ftr) 
        return self.node_ftr

    def get_WL_inputs(self, nonimg):
        '''get WL inputs for wl-deepgcn 
        '''
        # construct edge network inputs 
        n = self.node_ftr.shape[0] 
        num_edge = n*(1+n)//2 - n  # n*(n-1)//2,HO=6105
        pd_ftr_dim = nonimg.shape[1]
        edge_index = np.zeros([2, num_edge], dtype=np.int64) 
        edgenet_input = np.zeros([num_edge, 2*pd_ftr_dim], dtype=np.float32)  
        aff_score = np.zeros(num_edge, dtype=np.float32)
        # static affinity score used to pre-prune edges 
        aff_adj = get_static_affinity_adj(self.node_ftr, self.pd_dict)  
        flatten_ind = 0 
        for i in range(n):
            for j in range(i+1, n):
                edge_index[:,flatten_ind] = [i,j]
                edgenet_input[flatten_ind]  = np.concatenate((nonimg[i], nonimg[j]))
                aff_score[flatten_ind] = aff_adj[i][j]  
                flatten_ind +=1

        assert flatten_ind == num_edge, "Error in computing edge input"
        
        keep_ind = np.where(aff_score > 1.1)[0]  
        edge_index = edge_index[:, keep_ind]
        edgenet_input = edgenet_input[keep_ind]

        return edge_index, edgenet_input
    

In [55]:

import torch
from torch.nn import Linear as Lin, Sequential as Seq
import torch.nn.functional as F
from torch import nn

class WL(torch.nn.Module):
    def __init__(self, input_dim, dropout=0.3):
        super(WL, self).__init__()
        h1=256
        h2=128
        self.parser =nn.Sequential(
                nn.Linear(input_dim, h1, bias=True),
                nn.LeakyReLU(inplace=True),
                nn.BatchNorm1d(h1),
                nn.Dropout(dropout),
                nn.Linear(h1, h2, bias=True),
                nn.LeakyReLU(inplace=True),
                nn.BatchNorm1d(h2),
                nn.Dropout(dropout),
                nn.Linear(h2, h2, bias=True),
                )
        self.cos = nn.CosineSimilarity(dim=1, eps=1e-8)
        self.input_dim = input_dim
        self.model_init()
        self.relu = nn.ReLU(inplace=True)
        self.elu = nn.ReLU()

    def forward(self, x):
        x1 = x[:,0:self.input_dim]
        x2 = x[:,self.input_dim:]
        h1 = self.parser(x1) 
        h2 = self.parser(x2) 
        p = (self.cos(h1,h2) + 1)*0.5
        return p

    def model_init(self):
        for m in self.modules():
            if isinstance(m, Lin):
                torch.nn.init.kaiming_normal_(m.weight)
                m.weight.requires_grad = True
                if m.bias is not None:
                    m.bias.data.zero_()
                    m.bias.requires_grad = True



In [56]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.nn import Linear as Lin, Sequential as Seq
import torch_geometric as tg



class MLP(nn.Module):
    def __init__(self, input_dim, nhid):
        super(MLP,self).__init__()
        self.cls = nn.Sequential(
            torch.nn.Linear(input_dim,nhid))
        
    def forward(self, features):
        output = self.cls(features)
        return output
            
class GCN(nn.Module):
    def __init__(self, input_dim, nhid, num_classes, ngl, dropout, edge_dropout, edgenet_input_dim):
        super(GCN, self).__init__()
        print("phele init mai")
        K=3   
        hidden = [nhid for i in range(ngl)] 
        self.dropout = dropout
        self.edge_dropout = edge_dropout 
        bias = False 
        self.relu = torch.nn.ReLU(inplace=True) 
        self.ngl = ngl 
        self.gconv = nn.ModuleList()
        for i in range(ngl):
            in_channels = input_dim if i==0  else hidden[i-1]
            print(i," -- ",in_channels)
            self.gconv.append(tg.nn.ChebConv(in_channels, hidden[i], K, normalization='sym', bias=bias)) 
        print("start")  
        self.cls = nn.Sequential(
                torch.nn.Linear(16, 128),
                torch.nn.ReLU(inplace=True),
                nn.BatchNorm1d(128), 
                torch.nn.Linear(128, num_classes))
        print("end")
        self.edge_net = WL(input_dim=edgenet_input_dim//2, dropout=dropout)
        self.model_init()

    def model_init(self):
        print("welecome in model_init")
        for m in self.modules():
            if isinstance(m, Lin):
                torch.nn.init.kaiming_normal_(m.weight) # He init
                m.weight.requires_grad = True
                if m.bias is not None:
                    m.bias.data.zero_()
                    m.bias.requires_grad = True

    def forward(self, features, edge_index, edgenet_input, enforce_edropout=False): 
        print("welcome in forward")
        if self.edge_dropout>0:
            if enforce_edropout or self.training:
                one_mask = torch.ones([edgenet_input.shape[0],1])
                self.drop_mask = F.dropout(one_mask, self.edge_dropout, True)
                self.bool_mask = torch.squeeze(self.drop_mask.type(torch.bool))
                edge_index = edge_index[:, self.bool_mask] 
                edgenet_input = edgenet_input[self.bool_mask] # Weights
            
        edge_weight = torch.squeeze(self.edge_net(edgenet_input))
        

        # GCN residual connection
        # input layer
        print("welcome in input layer")
        features = F.dropout(features, self.dropout, self.training)
        x = self.relu(self.gconv[0](features, edge_index, edge_weight)) 
        x_temp = x
        
        # hidden layers
        print("hidden layer")
        for i in range(1, self.ngl - 1): # self.ngl→7
            x = F.dropout(x_temp, self.dropout, self.training)
            x = self.relu(self.gconv[i](x, edge_index, edge_weight)) 
            x_temp = x_temp + x # ([871,64])

        # output layer
        print("output layer")
        x = F.dropout(x_temp, self.dropout, self.training)
        x = self.relu(self.gconv[self.ngl - 1](x, edge_index, edge_weight))
        x_temp = x_temp + x

        output = x # Final output is not cumulative
        output = self.cls(output) 
        print(output ,  edge_weight)        
        return output, edge_weight
    

    



In [57]:
import pandas as pd
import numpy as np

In [58]:
import torch
import torchmetrics
from torchmetrics.classification import MulticlassSpecificity
import matplotlib.pyplot as plt
import numpy as np
import itertools

from sklearn.metrics import precision_recall_fscore_support

def torchmetrics_accuracy(preds, labels):
    acc = torchmetrics.functional.accuracy(preds, labels,task="multiclass", num_classes=2)
    return acc

def torchmetrics_spef(preds, labels):
    metric = MulticlassSpecificity(num_classes=2)
    spef = metric(preds, labels)
    return spef

def torchmetrics_auc(preds, labels):
    auc = torchmetrics.functional.auroc(preds, labels, task="multiclass", num_classes=2)
    return auc

def confusion_matrix(preds, labels):
    conf_matrix = torch.zeros(2, 2)
    preds = torch.argmax(preds, 1)
    for p, t in zip(preds, labels):
        conf_matrix[t, p] += 1 
    return conf_matrix
def plot_confusion_matrix(cm, normalize=False, title='Confusion matrix', cmap=plt.cm.Oranges):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    Input
    - cm : computer the value of confusion matrix
    - normalize : True: %, False: 123
    """
    classes = ['0:ASD','1:TC']
    if normalize:
        cm = cm.numpy()
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = '.2f' if normalize else '.0f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
def correct_num(preds, labels):
    """Accuracy, auc with masking.Acc of the masked samples"""
    correct_prediction = np.equal(np.argmax(preds, 1), labels).astype(np.float32)
    return np.sum(correct_prediction)

def prf(preds, labels, is_logit=True):
    ''' input: logits, labels  ''' 
    pred_lab= np.argmax(preds, 1)
    p,r,f,s  = precision_recall_fscore_support(labels, pred_lab, average='binary')
    return [p,r,f]







In [59]:
import argparse
parser = argparse.ArgumentParser()
args, unknown = parser.parse_known_args()

In [60]:
from __future__ import division
from __future__ import print_function

import os
import time
import argparse
import numpy as np
import io
import sys

import torch
import torch.optim as optim



# from dataloader import dataloader

from sklearn.model_selection import train_test_split
from sklearn.metrics import auc

if hasattr(sys.stdout, 'buffer'):
    sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8')


class Args:
    def __init__(self):
        self.no_cuda = False
        self.seed = 46
        self.epochs = 10
        self.lr = 0.001
        self.weight_decay = 5e-5
        self.hidden = 16
        self.dropout = 0.2
        self.atlas = 'ez'
        self.num_features = 2000
        self.folds = 2
        self.connectivity = 'correlation'
        self.max_degree = 3
        self.ngl = 8
        self.edropout = 0.3
        self.train = 1
        self.ckpt_path = '../folds/rois_ez_pth_b'
        self.early_stopping = True
        self.early_stopping_patience = 20

# Instantiate Args class
args = Args()

# Check if CUDA is available
args.cuda = not args.no_cuda and torch.cuda.is_available()

# Set random seeds
np.random.seed(args.seed)
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

# Create params dictionary
params = vars(args)

# Print Hyperparameters
print('Hyperparameters:')
for key, value in params.items():
    print(key + ":", value)

corrects = np.zeros(args.folds, dtype=np.int32) 
accs = np.zeros(args.folds, dtype=np.float32) 
aucs = np.zeros(args.folds, dtype=np.float32)
prfs = np.zeros([args.folds,3], dtype=np.float32) # Save Precision, Recall, F1
test_num = np.zeros(args.folds, dtype=np.float32)


print('  Loading dataset ...')
dataloader = dataloader()
raw_features, y, nonimg = dataloader.load_data(params) 
cv_splits = dataloader.data_split(args.folds)

features=raw_features

t1 = time.time()
count=1;
for i in range(args.folds):
    
    
    
    t_start = time.time()
    train_ind, test_ind = cv_splits[i]

    train_ind, valid_ind = train_test_split(train_ind, test_size=0.1, random_state = 24)
    
    cv_splits[i] = (train_ind, valid_ind)
    cv_splits[i] = cv_splits[i] + (test_ind,)
    print('Size of the {}-fold Training, Validation, and Test Sets:{},{},{}' .format(i+1, len(cv_splits[i][0]), len(cv_splits[i][1]), len(cv_splits[i][2])))

    if args.train == 1:
        for j in range(args.folds):
            print(' Starting the {}-{} Fold:：'.format(i+1,j+1))
            node_ftr = dataloader.get_node_features(train_ind)
            edge_index, edgenet_input = dataloader.get_WL_inputs(nonimg)
            edgenet_input = (edgenet_input - edgenet_input.mean(axis=0)) / edgenet_input.std(axis=0)
            
            model = GCN(input_dim = args.num_features,
                        nhid = args.hidden, 
                        num_classes = 2, 
                        ngl = args.ngl, 
                        dropout = args.dropout, 
                        edge_dropout = args.edropout, 
                        edgenet_input_dim = 2*nonimg.shape[1])
            optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
            
#             if args.cuda:
            model
            features = torch.tensor(node_ftr, dtype=torch.float32)
            edge_index = torch.tensor(edge_index, dtype=torch.long)
            edgenet_input = torch.tensor(edgenet_input, dtype=torch.float32)
            labels = torch.tensor(y, dtype=torch.long)
            fold_model_path = args.ckpt_path + "/fold{}.pth".format(i+1)
                
            acc = 0
            best_val_loss = float('inf') # early stoppping: Initialized to positive infinity
            current_patience = 0 # early stopping: Used to record the epochs of the current early stopping
            
            epoch_store = []
            acc_train_store =[]        
            pre_train_store =[]
            recall_train_store =[]
            F1_train_store =[]
            AUC_train_store =[]
            acc_val_store=[]
            pre_val_store=[]
            recall_val_store=[]
            F1_val_store=[]
            AUC_val_store=[]
            
            for epoch in range(args.epochs):
                # train
                model.train()
                with torch.set_grad_enabled(True):
                    optimizer.zero_grad()
                    output, edge_weights = model(features, edge_index, edgenet_input)
                    loss_train = torch.nn.CrossEntropyLoss()(output[train_ind], labels[train_ind])
                    loss_train.backward()
                    optimizer.step()
                acc_train = torchmetrics_accuracy(output[train_ind], labels[train_ind])
                auc_train = torchmetrics_auc(output[train_ind], labels[train_ind])
                logits_train = output[train_ind].detach().cpu().numpy()
                prf_train = prf(logits_train, y[train_ind])

                
                # valid
                model.eval()
                with torch.set_grad_enabled(False):
                    output, edge_weights = model(features, edge_index, edgenet_input)
                loss_val = torch.nn.CrossEntropyLoss()(output[valid_ind], labels[valid_ind])
                acc_val = torchmetrics_accuracy(output[valid_ind], labels[valid_ind])
                auc_val = torchmetrics_auc(output[valid_ind], labels[valid_ind])
                logits_val = output[valid_ind].detach().cpu().numpy()
                prf_val = prf(logits_val, y[valid_ind])

                
                print('Epoch:{:04d}'.format(epoch+1))
                print('acc_train:{:.4f}'.format(acc_train),
                      'pre_train:{:.4f}'.format(prf_train[0]),
                      'recall_train:{:.4f}'.format(prf_train[1]),
                      'F1_train:{:.4f}'.format(prf_train[2]),
                      'AUC_train:{:.4f}'.format(auc_train))
                print('acc_val:{:.4f}'.format(acc_val),
                      'pre_val:{:.4f}'.format(prf_val[0]),
                      'recall_val:{:.4f}'.format(prf_val[1]),
                      'F1_val:{:4f}'.format(prf_val[2]),
                      'AUC_val:{:.4f}'.format(auc_val))
                
                epoch_store.append(epoch+1)
                acc_train_store.append(acc_train)       
                pre_train_store.append(prf_train[0])
                recall_train_store.append(prf_train[1])
                F1_train_store.append(prf_train[2])
                AUC_train_store.append(auc_train)
                acc_val_store.append(acc_val)
                pre_val_store.append(prf_val[0])
                recall_val_store.append(prf_val[1])
                F1_val_store.append(prf_val[2])
                AUC_val_store.append(auc_val)
                
                # save pth
                if acc_val > acc and epoch > 50:
                    acc = acc_val
                    if args.ckpt_path != '':
                        if not os.path.exists(args.ckpt_path):
                            os.makedirs(args.ckpt_path)
                        torch.save(model.state_dict(), fold_model_path)
                
                # Early Stopping
                if epoch > 50 and args.early_stopping == True:
                    if loss_val < best_val_loss:
                        best_val_loss = loss_val
                        current_patience = 0
                    else:
                        current_patience += 1
                    if current_patience >= args.early_stopping_patience:
                        print('Early Stopping!!! epoch：{}'.format(epoch))
                        break
        print("===================================================================",i,"_",j)
        data  = { 
              "epoch" : epoch_store ,
              "acc_train" : acc_train_store ,        
              "pre_train" : pre_train_store ,
              "recall_train" : recall_train_store ,
              "F1_train" : F1_train_store ,
              "AUC_train" : AUC_train_store ,
              "acc_val" : acc_val_store,
               "pre_val" : pre_val_store ,
              "recall_val" : recall_val_store ,
              "F1_val" : F1_val_store ,
              "AUC_val" : AUC_val_store  
        }
        
        
#         epoch_file_path =  f'../files/rois_ez/file_{i}_{j}_{count}.csv'
#         data_file = pd.DataFrame(data);
#         data_file.to_csv(epoch_file_path , index=False);
#         count=count+1;
        # test
        print("Loading the Model for the {}-th Fold:... ...".format(i+1),
              "Size of samples in the test set:{}".format(len(test_ind)))
        model.load_state_dict(torch.load(fold_model_path))
        model.eval()
        
        with torch.set_grad_enabled(False):
            output, edge_weights = model(features, edge_index, edgenet_input)
        acc_test = torchmetrics_accuracy(output[test_ind], labels[test_ind])
        auc_test = torchmetrics_auc(output[test_ind], labels[test_ind])
        logits_test = output[test_ind].detach().cpu().numpy()
        correct_test = correct_num(logits_test, y[test_ind])
        prf_test =  prf(logits_test, y[test_ind])
        
        t_end = time.time()
        t = t_end - t_start
        print('Fold {} Results:'.format(i+1),
              'test acc:{:.4f}'.format(acc_test),
              'test_pre:{:.4f}'.format(prf_test[0]),
              'test_recall:{:.4f}'.format(prf_test[1]),
              'test_F1:{:.4f}'.format(prf_test[2]),
              'test_AUC:{:.4f}'.format(auc_test),
              'time:{:.3f}s'.format(t))
        
        correct = correct_test
        aucs[i] = auc_test
        prfs[i] = prf_test
        corrects[i] = correct
        test_num[i] = len(test_ind)
    
    
    if args.train == 0:
        node_ftr = dataloader.get_node_features(train_ind)
        edge_index, edgenet_input = dataloader.get_WL_inputs(nonimg)
        edgenet_input = (edgenet_input - edgenet_input.mean(axis=0)) / edgenet_input.std(axis=0)
        
        model = GCN(input_dim = args.num_features,
                    nhid = args.hidden, 
                    num_classes = 2, 
                    ngl = args.ngl, 
                    dropout = args.dropout, 
                    edge_dropout = args.edropout, 
                    edgenet_input_dim = 2*nonimg.shape[1])
        optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
        
#         if args.cuda
        model
        features = torch.tensor(node_ftr, dtype=torch.float)
        edge_index = torch.tensor(edge_index, dtype=torch.long)
        edgenet_input = torch.tensor(edgenet_input, dtype=torch.float32)
        labels = torch.tensor(y, dtype=torch.long)
        fold_model_path = args.ckpt_path + "/fold{}.pth".format(i+1)
        
        model.load_state_dict(torch.load(fold_model_path))
        model.eval()
        
        with torch.set_grad_enabled(False):
            output, edge_weights = model(features, edge_index, edgenet_input)
        acc_test = torchmetrics_accuracy(output[test_ind], labels[test_ind])
        auc_test = torchmetrics_auc(output[test_ind], labels[test_ind])
        logits_test = output[test_ind].detach().cpu().numpy()
        correct_test = correct_num(logits_test, y[test_ind])
        prf_test =  prf(logits_test, y[test_ind])
        
        t_end = time.time()
        t = t_end - t_start
        print('Fold {} Results:'.format(i+1),
              'test acc:{:.4f}'.format(acc_test),
              'test_pre:{:.4f}'.format(prf_test[0]),
              'test_recall:{:.4f}'.format(prf_test[1]),
              'test_F1:{:.4f}'.format(prf_test[2]),
              'test_AUC:{:.4f}'.format(auc_test),
              'time:{:.3f}s'.format(t))
        
        correct = correct_test
        aucs[i] = auc_test
        prfs[i] = prf_test
        corrects[i] = correct
        test_num[i] = len(test_ind)

t2 = time.time()

print('\r\n======Finish Results for Nested 10-fold cross-validation======')
Nested10kCV_acc = np.sum(corrects) / np.sum(test_num)
Nested10kCV_auc = np.mean(aucs)
Nested10kCV_precision, Nested10kCV_recall, Nested10kCV_F1 = np.mean(prfs, axis=0)
print('Test:',
      'acc:{}'.format(Nested10kCV_acc),
      'precision:{}'.format(Nested10kCV_precision),
      'recall:{}'.format(Nested10kCV_recall),
      'F1:{}'.format(Nested10kCV_F1),
      'AUC:{}'.format(Nested10kCV_auc))
print('Total duration:{}'.format(t2 - t1))



Hyperparameters:
no_cuda: False
seed: 46
epochs: 10
lr: 0.001
weight_decay: 5e-05
hidden: 16
dropout: 0.2
atlas: ez
num_features: 2000
folds: 2
connectivity: correlation
max_degree: 3
ngl: 8
edropout: 0.3
train: 1
ckpt_path: ../folds/rois_ez_pth_b
early_stopping: True
early_stopping_patience: 20
cuda: False
  Loading dataset ...


C:\Users\KSB\AppData\Local\Temp\ipykernel_2032\4253426024.py:218: RuntimeWarning: divide by zero encountered in arctanh
  norm_networks = [np.arctanh(mat) if not np.all(np.abs(mat) == 1) else mat for mat in all_networks]


skf :  StratifiedKFold(n_splits=2, random_state=None, shuffle=False)
Size of the 1-fold Training, Validation, and Test Sets:500,56,556
 Starting the 1-1 Fold:：
Fitting estimator with 4656 features.
Fitting estimator with 4556 features.
Fitting estimator with 4456 features.
Fitting estimator with 4356 features.
Fitting estimator with 4256 features.
Fitting estimator with 4156 features.
Fitting estimator with 4056 features.
Fitting estimator with 3956 features.
Fitting estimator with 3856 features.
Fitting estimator with 3756 features.
Fitting estimator with 3656 features.
Fitting estimator with 3556 features.
Fitting estimator with 3456 features.
Fitting estimator with 3356 features.
Fitting estimator with 3256 features.
Fitting estimator with 3156 features.
Fitting estimator with 3056 features.
Fitting estimator with 2956 features.
Fitting estimator with 2856 features.
Fitting estimator with 2756 features.
Fitting estimator with 2656 features.
Fitting estimator with 2556 features.
Fitt

welcome in input layer
hidden layer
output layer
tensor([[ 0.4437, -0.6517],
        [ 0.2119, -0.1788],
        [ 0.6277, -0.3773],
        ...,
        [ 0.7843, -0.3126],
        [ 0.2980, -0.6912],
        [ 0.9487, -0.3802]], grad_fn=<AddmmBackward0>) tensor([0.6565, 0.5817, 0.5389,  ..., 0.6978, 0.7676, 0.7022],
       grad_fn=<SqueezeBackward0>)
welcome in forward
welcome in input layer
hidden layer
output layer
tensor([[ 0.1787, -0.1002],
        [ 0.1705, -0.0949],
        [ 0.1694, -0.0917],
        ...,
        [ 0.1717, -0.0937],
        [ 0.1745, -0.1010],
        [ 0.1729, -0.1042]]) tensor([0.8082, 0.7037, 0.5938,  ..., 0.9797, 0.9340, 0.8447])
Epoch:0009
acc_train:0.6840 pre_train:0.7297 recall_train:0.6231 F1_train:0.6722 AUC_train:0.7295
acc_val:0.8036 pre_val:0.8333 recall_val:0.7407 F1_val:0.784314 AUC_val:0.8314
welcome in forward
welcome in input layer
hidden layer
output layer
tensor([[ 0.3452, -0.3379],
        [ 0.5369, -0.3754],
        [ 1.3657, -0.1921],
   

C:\Users\KSB\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

C:\Users\KSB\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


hidden layer
output layer
tensor([[ 0.0300, -0.6459],
        [ 0.4036,  0.0843],
        [ 0.3603, -1.5889],
        ...,
        [-0.1808,  0.2828],
        [ 1.0418, -0.8661],
        [ 0.6910, -0.6123]], grad_fn=<AddmmBackward0>) tensor([0.6930, 0.6262, 0.5536,  ..., 0.8390, 0.7009, 0.7082],
       grad_fn=<SqueezeBackward0>)
welcome in forward
welcome in input layer
hidden layer
output layer
tensor([[ 0.2245, -0.0223],
        [ 0.1930, -0.0184],
        [ 0.1878, -0.0207],
        ...,
        [ 0.1150, -0.0247],
        [ 0.2496, -0.0175],
        [ 0.1655, -0.0251]]) tensor([0.6467, 0.5742, 0.4239,  ..., 0.9946, 0.9676, 0.9354])
Epoch:0004
acc_train:0.6100 pre_train:0.6117 recall_train:0.6846 F1_train:0.6461 AUC_train:0.6858
acc_val:0.5000 pre_val:0.3333 recall_val:0.0370 F1_val:0.066667 AUC_val:0.3512
welcome in forward
welcome in input layer
hidden layer
output layer
tensor([[ 0.0398, -3.5150],
        [ 1.2396, -0.8540],
        [-0.0764, -0.5226],
        ...,
        [ 1.0

welcome in forward
welcome in input layer
hidden layer
output layer
tensor([[ 0.0010,  0.0181],
        [-0.0084,  0.0089],
        [-0.0058,  0.0108],
        ...,
        [-0.0145,  0.0043],
        [ 0.0019,  0.0140],
        [-0.0065,  0.0113]]) tensor([0.6641, 0.5815, 0.5747,  ..., 0.9990, 0.9936, 0.9874])
Epoch:0001
acc_train:0.5040 pre_train:0.5309 recall_train:0.3962 F1_train:0.4537 AUC_train:0.5058
acc_val:0.4464 pre_val:0.4390 recall_val:0.6923 F1_val:0.537313 AUC_val:0.3923
welcome in forward
welcome in input layer
hidden layer
output layer
tensor([[-0.1898, -0.5184],
        [-0.1639, -0.3310],
        [-0.4258,  0.3434],
        ...,
        [-1.3482,  1.3675],
        [-0.2268, -0.5480],
        [-0.1805, -0.4697]], grad_fn=<AddmmBackward0>) tensor([0.6791, 0.4807, 0.7850,  ..., 0.8055, 0.7414, 0.7547],
       grad_fn=<SqueezeBackward0>)
welcome in forward
welcome in input layer
hidden layer
output layer
tensor([[-0.0235,  0.0135],
        [-0.0286,  0.0100],
        [-0.

Fitting estimator with 4456 features.
Fitting estimator with 4356 features.
Fitting estimator with 4256 features.
Fitting estimator with 4156 features.
Fitting estimator with 4056 features.
Fitting estimator with 3956 features.
Fitting estimator with 3856 features.
Fitting estimator with 3756 features.
Fitting estimator with 3656 features.
Fitting estimator with 3556 features.
Fitting estimator with 3456 features.
Fitting estimator with 3356 features.
Fitting estimator with 3256 features.
Fitting estimator with 3156 features.
Fitting estimator with 3056 features.
Fitting estimator with 2956 features.
Fitting estimator with 2856 features.
Fitting estimator with 2756 features.
Fitting estimator with 2656 features.
Fitting estimator with 2556 features.
Fitting estimator with 2456 features.
Fitting estimator with 2356 features.
Fitting estimator with 2256 features.
Fitting estimator with 2156 features.
Fitting estimator with 2056 features.
Number of labeled samples 500
Number of features s

welcome in forward
welcome in input layer
hidden layer
output layer
tensor([[-0.0785,  0.0247],
        [-0.0783,  0.0259],
        [-0.0768,  0.0222],
        ...,
        [-0.0768,  0.0231],
        [-0.0761,  0.0211],
        [-0.0771,  0.0210]]) tensor([0.7777, 0.4104, 0.3810,  ..., 0.9821, 0.8721, 0.7674])
Epoch:0009
acc_train:0.5340 pre_train:0.5308 recall_train:0.8962 F1_train:0.6667 AUC_train:0.5370
acc_val:0.4643 pre_val:0.4643 recall_val:1.0000 F1_val:0.634146 AUC_val:0.5872
welcome in forward
welcome in input layer
hidden layer
output layer
tensor([[ 0.0362,  0.0245],
        [-0.0036,  0.0422],
        [ 0.0245,  0.0172],
        ...,
        [ 0.1995, -0.0631],
        [ 0.0291,  0.0671],
        [-0.0340,  0.1000]], grad_fn=<AddmmBackward0>) tensor([0.7313, 0.5379, 0.4908,  ..., 0.7779, 0.7943, 0.7283],
       grad_fn=<SqueezeBackward0>)
welcome in forward
welcome in input layer
hidden layer
output layer
tensor([[-0.0772,  0.0203],
        [-0.0774,  0.0215],
        [-0.

1112
